# Notebook 32 — End-to-End Pipeline Test (Processing + Inference)

## Purpose

This notebook is a thin test harness. It:
1. Loads a holdout player's opening stats from the local DB
2. Calls the single end-to-end pipeline entrypoint in `utils/foldin_data_processing/pipeline.py`
   - processing (same as notebook 29)
   - inference + recommendation ranking (same outputs as notebook 31)
3. Prints the top recommended openings and key summary stats

All meat-and-potatoes logic lives in `pipeline.py`. This notebook should mainly be config + one function call.

---

In [1]:
# Standard library imports
import sys
from pathlib import Path
import pickle

# Third-party imports
import pandas as pd

# Add repo root to path and ensure notebooks/ is on path (project code lives there)
PROJECT_ROOT = Path.cwd().parent
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(NOTEBOOKS_DIR))

# Sanity check: show where `utils` is being imported from
import importlib
utils_mod = importlib.import_module("utils")
print(f"utils imported from: {getattr(utils_mod, '__file__', utils_mod)}")

# Force reload in case the kernel has a stale definition of PipelineConfig/pipeline.py
import utils.foldin_data_processing.pipeline as pipeline_mod
pipeline_mod = importlib.reload(pipeline_mod)
print(f"pipeline module: {pipeline_mod.__file__}")

# Local imports (after reload)
from utils.database.db_utils import get_db_connection
from utils.foldin_data_processing.types import PlayerData
from utils.foldin_data_processing.pipeline import (
    PipelineConfig,
    PipelineArtifacts,
    run_foldin_pipeline,
 )

print("✓ All imports successful")

utils imported from: /Users/a/Documents/personalprojects/chess-opening-recommender/notebooks/utils/__init__.py
pipeline module: /Users/a/Documents/personalprojects/chess-opening-recommender/notebooks/utils/foldin_data_processing/pipeline.py
✓ All imports successful
pipeline module: /Users/a/Documents/personalprojects/chess-opening-recommender/notebooks/utils/foldin_data_processing/pipeline.py
✓ All imports successful


## Configuration

In [2]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Color to test ('w' or 'b') - change THIS ONE LINE
COLOR = 'b'

# Model directory name (ternary so it follows COLOR)
MODEL_DIR_NAME = "20251111_155428_white" if COLOR == 'w' else "20251212_152017_black"

# Holdout player index (from holdout_players.csv)
HOLDOUT_PLAYER_INDEX = 0

# Pipeline verbosity (0=silent, 1=progress, 2=detailed)
VERBOSE = 2

# How many openings to return
TOP_N_RECOMMENDATIONS = 50

# ============================================================================

COLOR_NAME = "White" if COLOR == 'w' else "Black"

print("═" * 80)
print("TEST CONFIGURATION")
print("═" * 80)
print(f"Color: {COLOR_NAME} ('{COLOR}')")
print(f"Model directory: {MODEL_DIR_NAME}")
print(f"Holdout player index: {HOLDOUT_PLAYER_INDEX}")
print(f"Verbosity: {VERBOSE}")
print(f"Top-N recommendations: {TOP_N_RECOMMENDATIONS}")
print("═" * 80)

════════════════════════════════════════════════════════════════════════════════
TEST CONFIGURATION
════════════════════════════════════════════════════════════════════════════════
Color: Black ('b')
Model directory: 20251212_152017_black
Holdout player index: 0
Verbosity: 2
Top-N recommendations: 50
════════════════════════════════════════════════════════════════════════════════


## Setup Paths

In [3]:
# Paths
DB_PATH = PROJECT_ROOT / "data" / "processed" / "chess_games.db"
MODEL_ARTIFACTS_DIR = PROJECT_ROOT / "data" / "models" / MODEL_DIR_NAME

# Verify paths exist
print("Verifying paths...")
print(f"  Database: {DB_PATH.exists()} - {DB_PATH}")
print(f"  Model artifacts: {MODEL_ARTIFACTS_DIR.exists()} - {MODEL_ARTIFACTS_DIR}")

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found: {DB_PATH}")
if not MODEL_ARTIFACTS_DIR.exists():
    raise FileNotFoundError(f"Model artifacts not found: {MODEL_ARTIFACTS_DIR}")

print("✓ All paths verified")

Verifying paths...
  Database: True - /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/chess_games.db
  Model artifacts: True - /Users/a/Documents/personalprojects/chess-opening-recommender/data/models/20251212_152017_black
✓ All paths verified


## Create Pipeline Config

In [4]:
# Create pipeline configuration
config = PipelineConfig(
    model_artifacts_dir=MODEL_ARTIFACTS_DIR,
    color=COLOR,
    min_games_threshold=3,
    k_shrinkage=50,
    top_n_recommendations=TOP_N_RECOMMENDATIONS,
    inference_batch_size=512,
    device='auto',
    verbose=VERBOSE,
 )

print(f"✓ Pipeline config created")
print(f"  Min games threshold: {config.min_games_threshold}")
print(f"  Bayesian shrinkage K: {config.k_shrinkage}")

✓ Pipeline config created
  Min games threshold: 3
  Bayesian shrinkage K: 50


## Load Holdout Player Data from Database

We'll load the same player data that notebook 29 used, so we can verify the pipeline produces identical results.

In [5]:
# Load holdout player metadata
holdout_players_path = MODEL_ARTIFACTS_DIR / "holdout_players.csv"

print(f"Loading holdout players from: {holdout_players_path}")
if not holdout_players_path.exists():
    raise FileNotFoundError(f"holdout_players.csv not found: {holdout_players_path}")

# NOTE: This environment is hitting a pandas/numpy bug where even constructing DataFrames triggers:
#   TypeError: Cannot convert numpy.ndarray to numpy.ndarray
# So we avoid pandas entirely for this step and parse the CSV with the stdlib only.
import csv

def _load_holdout_players_rows(path: Path) -> list[dict]:
    with path.open("r", encoding="utf-8", errors="replace", newline="") as f:
        reader = csv.DictReader(f)
        rows = []
        for row in reader:
            rows.append({k: (v if v is not None else "") for k, v in row.items()})
    if not rows:
        raise ValueError(f"holdout_players.csv is empty: {path}")
    return rows

rows = _load_holdout_players_rows(holdout_players_path)

if HOLDOUT_PLAYER_INDEX < 0 or HOLDOUT_PLAYER_INDEX >= len(rows):
    raise IndexError(
        f"HOLDOUT_PLAYER_INDEX={HOLDOUT_PLAYER_INDEX} out of range for {len(rows)} holdout players"
    )

row = rows[HOLDOUT_PLAYER_INDEX]
for required in ("db_id", "name", "rating"):
    if required not in row:
        raise ValueError(f"holdout_players.csv missing required column '{required}'")

test_player_db_id = int(row["db_id"])
test_player_name = str(row["name"])
test_player_rating = int(row["rating"])

print(f"Selected holdout player #{HOLDOUT_PLAYER_INDEX}:")
print(f"  Name: {test_player_name}")
print(f"  Database ID: {test_player_db_id}")
print(f"  Rating: {test_player_rating}")

Loading holdout players from: /Users/a/Documents/personalprojects/chess-opening-recommender/data/models/20251212_152017_black/holdout_players.csv
Selected holdout player #0:
  Name: AAashraf
  Database ID: 53
  Rating: 1723


In [6]:
# Extract player's opening statistics from database
conn = get_db_connection(DB_PATH)

query = """
SELECT 
    pos.player_id,
    pos.opening_id,
    o.eco,
    o.name as opening_name,
    pos.num_wins + pos.num_draws + pos.num_losses as num_games,
    pos.num_wins,
    pos.num_draws,
    pos.num_losses
FROM player_opening_stats pos
JOIN opening o ON pos.opening_id = o.id
WHERE pos.player_id = ?
  AND pos.color = ?
  AND (pos.num_wins + pos.num_draws + pos.num_losses) > 0
ORDER BY (pos.num_wins + pos.num_draws + pos.num_losses) DESC
"""

opening_stats_df = conn.execute(query, [int(test_player_db_id), COLOR]).df()
conn.close()

# Create PlayerData object
player_data = PlayerData(
    player_id=test_player_db_id,
    name=test_player_name,
    rating=test_player_rating,
    color=COLOR,
    opening_stats_df=opening_stats_df
)

print(f"\n✓ Loaded raw player data from database")
print(f"  Openings: {len(opening_stats_df)}")
print(f"  Total games: {player_data.total_games():,}")
print(f"  Mean score: {player_data.mean_score():.4f}")


✓ Loaded raw player data from database
  Openings: 194
  Total games: 3,502
  Mean score: 0.4604


## Run the Pipeline (Processing + Inference)

One function call: it will process the player into model-ready inputs, load the trained model, score all openings, and return the top-N recommendations.

In [7]:
# End-to-end pipeline (returns recommendations + summary)
result = run_foldin_pipeline(player_data=player_data, config=config)

print("\n" + "═" * 80)
print("PIPELINE OUTPUT (TOP RECOMMENDATIONS)")
print("═" * 80)
print(f"Color: {result['player']['color']}")
print(f"Rating Z: {result['player']['rating_z']:.4f}")
print(f"Total openings scored: {result['stats']['num_openings_total']:,}")
print(f"Openings played: {result['stats']['num_openings_played']:,}")
print(f"Openings unplayed: {result['stats']['num_openings_unplayed']:,}")
print(f"Top-N: {result['stats']['top_n']}")
print("")
for i, rec in enumerate(result['recommendations'], 1):
    print(f"{i:2d}. {rec['eco']:<4} {rec['opening_name']:<60} ({rec['predicted_score']:.4f})")
print("═" * 80)

✓ Loaded artifacts for black openings
  • 2,728 valid openings
  • Rating norm: mean=1765, std=249
  • Opening stats: 2,728 entries
  • Hyperparams: players=48,551, openings=2,728, factors=40

Processing Black openings for: AAashraf
  Rating: 1723
  Input openings: 194
  Total games: 3,502

STEP: Filtering Valid Openings
  Original: 194 openings, 3,502 games
  Filtered: 94 openings, 3,362 games

STEP: Calculating Raw Scores
  Range: [0.0000, 1.0000]
  Mean: 0.5035

STEP: Remapping Opening IDs
  Training ID range: [10, 2627]

STEP: Normalizing Player Rating
  1723 → -0.1696

STEP: Applying Bayesian Shrinkage
  Mean adjustment: -0.0331
  Adjustment range: [-0.5596, +0.4993]

STEP: Encoding ECO Features
  Encoded 94 ECO codes

STEP: Creating Model Input
  Shape: (94,)
  Player: AAashraf
  Rating Z: -0.1696

✓ Pipeline complete
  Output openings: 94
  Ready for inference

Running inference...
Loading inference model...
  Training ID range: [10, 2627]

STEP: Normalizing Player Rating
  1723

## Validation: Compare with Saved Data from Notebook 29

Load the saved output from notebook 29 and verify our pipeline produces identical results.